In [1]:
import numpy as np
from numpy.linalg import inv, det
import matplotlib.pyplot as plt
from copy import copy
import random
%matplotlib inline 

## Парето

In [2]:
def find_pareto_optimal(game_matrix):
    optimal_strats = []
    
    def is_optimal(row, col):
        input_item = game_matrix[row, col]
        
        for i in range(game_matrix.shape[0]):
            for j in range(game_matrix.shape[1]):
                cur_item = game_matrix[i, j]

                if (input_item <= cur_item).all() and(input_item < cur_item).any():
                    return False

        return True
    
    for i in range(game_matrix.shape[0]):
        for j in range(game_matrix.shape[1]):
            if is_optimal(i, j):
                optimal_strats.append((i, j))
                
    return optimal_strats            

## Нэш

In [3]:
# массив индексов максимального элемента
def max_item_indexes(vector):
    indexes = np.linspace(0, vector.shape[0] - 1, vector.shape[0], dtype = int)
    
    return indexes[np.isin(vector, np.max(vector))]

def find_nash_optimal(game_matrix):
    # индексы наилучших стратегий первого игрока при i-й стратегии второго
    def first_max_ids(i):
        return max_item_indexes(game_matrix[:, i, 0])
    
    # индексы наилучших стратегий второго игрока при i-й стратегии первого
    def second_max_ids(i):
        return max_item_indexes(game_matrix[i, :, 1])
    
    optimal_strats = []
    
    for i in range(game_matrix.shape[0]):
        second_ids = second_max_ids(i)
        for j in second_ids:
            first_ids = first_max_ids(j)
            
            if i in first_ids:
                optimal_strats.append((i, j))
                    
    return optimal_strats

## Цветная печать

In [4]:
NASH_PREFIX = '\x1b[31m'
PARETO_PREFIX = '\x1b[32m'
BOTH_PREFIX = '\x1b[34m'
POSTFIX = '\x1b[0m'

def color_nash(string):
    return NASH_PREFIX + string + POSTFIX

def color_pareto(string):
    return PARETO_PREFIX + string + POSTFIX

def color_both(string):
    return BOTH_PREFIX + string + POSTFIX

def print_game_solution(game_matrix):
    print(color_nash('Nash optimal'))
    print(color_pareto('Pareto optimal'))
    print(color_both('Both optimal'))
    print()
    
    pareto = find_pareto_optimal(game_matrix)
    nash = find_nash_optimal(game_matrix)
    
    maxlen = 0
    
    for i in range(game_matrix.shape[0]):
        for j in range(game_matrix.shape[1]):
            maxlen = max(maxlen, len(str(game_matrix[i, j])))
    
    for i in range(game_matrix.shape[0]):
        row = []
        for j in range(game_matrix.shape[1]):
            item = (i, j)
            item_str = str(game_matrix[item])
            
            item_str = ' ' * (maxlen - len(item_str)) + item_str
            
            if item in pareto:
                if item in nash:
                    item_str = color_both(item_str)
                else:
                    item_str = color_pareto(item_str)
            elif item in nash:
                item_str = color_nash(item_str)

            row.append(item_str)
            
        print('  '.join(row))

## Условия

In [5]:
epsilon = round(random.randrange(10, 90)/100, 2)

crossroad = np.array(
   [[[1, 1], [1 - epsilon, 2]],
    [[2, 1 - epsilon], [0, 0]]]
)

family_conflict = np.array(
   [[[4, 1], [0, 0]],
    [[0, 0], [1, 4]]]
)

prisoners = np.array(
   [[[ -5, -5], [ 0, -10]],
    [[-10,  0], [-1,  -1]]]
)

var_15 = np.array(
   [[[0, 10], [9, 1]],
    [[7, 8], [6, 11]]]
)

In [6]:
print_game_solution(crossroad)

Nash optimal
Pareto optimal
Both optimal

    [1. 1.]  [0.31 2.  ]
[2.   0.31]      [0. 0.]


In [7]:
print_game_solution(family_conflict)

Nash optimal
Pareto optimal
Both optimal

[4 1]  [0 0]
[0 0]  [1 4]


In [8]:
print_game_solution(prisoners)

Nash optimal
Pareto optimal
Both optimal

  [-5 -5]  [  0 -10]
[-10   0]    [-1 -1]


In [9]:
print_game_solution(var_15)

Nash optimal
Pareto optimal
Both optimal

[ 0 10]    [9 1]
  [7 8]  [ 6 11]


In [10]:
print_game_solution(np.random.randint(-50, 50, (10, 10, 2)))

Nash optimal
Pareto optimal
Both optimal

[-15 -17]    [46 41]    [34 48]  [ 13 -11]    [43 42]    [10 28]  [-16 -26]  [-21 -28]  [ 27 -39]  [ 15 -25]
[-19 -32]    [-7 29]  [ 30 -15]  [ -8 -42]    [-4  7]    [45 15]  [ 30 -20]    [12 20]  [-50 -50]  [-25 -12]
  [29 14]    [26 21]  [  7 -19]  [-26 -23]    [45  4]  [-38  20]    [35 38]  [-30  36]    [31 40]    [38  5]
[ 17 -19]  [-36  48]    [ 2 35]  [-39  37]  [-15  49]    [-3 10]    [25 45]  [-22 -27]    [ 9 16]    [18 27]
[-44  19]    [-4 42]  [-41  -8]    [20 30]  [-13  42]  [  7 -23]  [-32  35]  [-31 -40]    [-5  0]  [-49  34]
  [ 6 17]    [35 26]  [-35  35]  [ 20 -47]  [ 45 -18]    [22 17]  [ 31 -26]    [32 -6]  [-44  40]  [ 37 -14]
[ 11 -27]  [ 13 -38]    [29  8]  [  9 -49]    [31 -4]  [-15  49]  [-33  42]  [-46  18]  [-10 -42]    [37 -5]
[-28 -22]    [ 9 47]    [23 -5]    [21 -7]    [38 25]    [26 20]  [-37 -49]    [36  0]    [ 9 -6]  [ 23 -41]
[ 35 -12]  [-22 -36]  [-13 -34]  [ -9 -35]  [-26  -6]    [45 29]  [-26  22]  [-29   9]

## Поиск смешанных стратегий

In [11]:
u = np.ones(2)

A = var_15[:,:,0]
B = var_15[:,:,1]

v1 = 1/(u.dot(inv(A)).dot(u))
v2 = 1/(u.dot(inv(B)).dot(u))

x = v2 * u.dot(inv(B))
y = v1 * u.dot(inv(A))

In [12]:
print('A:')
print(A)
print('B:')
print(B)
print()
print('v1:', round(v1, 3))
print('v2:', round(v2, 3))
print()
print('x:', x)
print('y:', y)

A:
[[0 9]
 [7 6]]
B:
[[10  1]
 [ 8 11]]

v1: 6.3
v2: 8.5

x: [0.25 0.75]
y: [0.1 0.9]
